In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math

from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv(
    "/Users/swastikagarwal/Downloads/SEM5_RVCE/LAB WORK/NLP/nlp dataset/spam.csv",encoding="Windows-1251")
# X=df['v2']
# y=df['v1']
X=df.iloc[:,1]
y=df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will М_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [3]:
def preprocess(text):
    stop_words=set(stopwords.words('english'))
    text=text.lower()
    words=word_tokenize(text)
    list1=[]

    for word in words:
        if word not in stop_words and word.isalnum():
            list1.append(word)
    
    list_doc=' '.join(list1)
    return list_doc

In [4]:
X_train=X_train.apply(preprocess)
X_test=X_test.apply(preprocess)

In [5]:
def dff(word,sentence):
    words=sentence.split()
    return words.count(word)/(len(words)+1)

def idff(word,sentences):
    n=0
    for sent in sentences:
        if word in sent:
            n+=1
    return math.log(len(sentences)/(n+1))

all_docs=X_train.to_list()+X_test.to_list()

unique_words=set(' '.join(all_docs).split())

idf_score={}
for word in unique_words:
    idf_score[word]=idff(word,all_docs)

#vocabulary
vocab=list(idf_score.keys())

X_train_vector=[]
for sentence in X_train:
    temp_list=[]
    for w in vocab:
        tfidf_score=dff(w,sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_train_vector.append(temp_list)


X_test_vector = []
for sentence in X_test:
    temp_list = []
    for w in vocab:
        tfidf_score = dff(w, sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_test_vector.append(temp_list)

In [6]:
X_train_vector=np.array(X_train_vector)
X_test_vector=np.array(X_test_vector)

In [7]:
# model=LogisticRegression(max_iter=100)
# model.fit(X_train_vector,y_train)

model=RandomForestClassifier()
model.fit(X_train_vector,y_train)

RandomForestClassifier()

In [8]:
text = "win To claim call 09061701461.."

text=preprocess(text)
vector=[]

for word in vocab:
    tfs=dff(word,text)
    tf_idf_score=tfs*idf_score[word]
    vector.append(tf_idf_score)

output=model.predict([vector])
output


array(['spam'], dtype=object)